In [71]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re
import sqlite3
import time
from datetime import datetime
from statistics import mean
import mysql.connector

In [3]:

prefix = 'https://www.espn.com/nfl/matchup/_/gameId/'
game_ids = []

for year in range(2006,2023):
    time.sleep(1)
    webpage_response = requests.get('https://www.espn.com/nfl/schedulegrid/_/year/'+str(year), headers={'User-Agent': 'Mozilla/5.0 (iPad; CPU OS 12_2 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Mobile/15E148'})
    webpage = webpage_response.content
    soup = BeautifulSoup(webpage, "html.parser")
    table = soup.find("table")
    all_links = table.find_all("a",href=True)
    for link in all_links:
        if len(link['href'].split("//www.espn.com/nfl/boxscore/_/gameId/")[-1]) == 9:
            game_ids.append(link['href'].split("//www.espn.com/nfl/boxscore/_/gameId/")[-1])

game_ids = [*set(game_ids)]


In [4]:
print(len(game_ids))

4383


In [168]:
mydb = mysql.connector.connect(
  host="127.0.0.1",
  user="root",
  password="vuQDE37idjRX",
  database="nfl"
)
cur = mydb.cursor()

In [115]:
for i in range(len(game_ids)):
    i=4383
    webpage_response = requests.get(prefix+game_ids[i], headers={'User-Agent': 'Mozilla/5.0 (iPad; CPU OS 12_2 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Mobile/15E148'})
    webpage = webpage_response.content
    soup = BeautifulSoup(webpage, "html.parser")
    
    teams = soup.find_all("h2")
    away_team = teams[0].get_text().split(" ")[-1]
    home_team = teams[1].get_text().split(" ")[-1]
    if away_team == "Redskins" or away_team == "Washington":
        away_team = "Commanders"
    elif home_team == "Redskins" or home_team == "Washington":
        home_team = "Commanders"

    scores = soup.find_all("div",{"class":"Gamestrip__Score relative tc w-100 fw-heavy h2 clr-gray-01"})
    away_score = re.split("W",scores[0].get_text(),1)[0]
    home_score = re.split("W",scores[1].get_text(),1)[0]
    if len(re.split("W",scores[0].get_text(),1)) == 2:
        away_w_l = 1
        home_w_l = 0
    else:
        away_w_l = 0
        home_w_l = 1

    if soup.find("div",{"class":"n6 clr-gray-03 GameInfo__Location__Name"}) == None:
        try:
            stadium = soup.find("div",{"class":"n6 clr-gray-03 GameInfo__Location__Name--noImg"}).get_text()
        except:
            print(i,game_ids[i],"No Stadium")
            continue
    else:
        stadium = soup.find("div",{"class":"n6 clr-gray-03 GameInfo__Location__Name"}).get_text()
    

    date_time = soup.find("div",{"class":"n8 GameInfo__Meta"}).get_text().split("Cove")[0]
    date_time = datetime.strptime(date_time, '%I:%M %p, %B %d, %Y')
    date = date_time.strftime("%Y-%m-%d") 
    hour = date_time.strftime("%H")

    cur.execute('SELECT latitude FROM stadiums WHERE name="'+stadium+'"')
    lat = cur.fetchone()[0]
    cur.execute('SELECT longitude FROM stadiums WHERE name="'+stadium+'"')
    long = cur.fetchone()[0]

    webpage_response = requests.get('https://archive-api.open-meteo.com/v1/archive?latitude='+str(lat)+'&longitude='+str(long)+'&start_date='+str(date)+'&end_date='+str(date)+'&hourly=temperature_2m,relativehumidity_2m,apparent_temperature,rain,snowfall,surface_pressure,cloudcover,windspeed_100m,winddirection_100m&temperature_unit=fahrenheit&windspeed_unit=mph&precipitation_unit=inch&timezone=America%2FNew_York')
    webpage_response = webpage_response.json()

    temp = mean(webpage_response['hourly']['temperature_2m'][int(hour):int(hour)+4])
    humidity = mean(webpage_response['hourly']['relativehumidity_2m'][int(hour):int(hour)+4])
    apparent_temp = mean(webpage_response['hourly']['apparent_temperature'][int(hour):int(hour)+4])
    rain = mean(webpage_response['hourly']['rain'][int(hour):int(hour)+4])
    snowfall = mean(webpage_response['hourly']['snowfall'][int(hour):int(hour)+4])
    surface_pressure = mean(webpage_response['hourly']['surface_pressure'][int(hour):int(hour)+4])
    cloudcover = mean(webpage_response['hourly']['cloudcover'][int(hour):int(hour)+4])
    windspeed_100m = mean(webpage_response['hourly']['windspeed_100m'][int(hour):int(hour)+4])
    winddirection_100m = mean(webpage_response['hourly']['winddirection_100m'][int(hour):int(hour)+4])
    weather_lst = [temp,humidity,apparent_temp,rain,snowfall,surface_pressure,cloudcover,windspeed_100m,winddirection_100m]

    temp_home = [i*2+1,game_ids[i],date,hour,1,home_team,stadium,home_score,home_w_l]
    temp_away = [i*2,game_ids[i],date,hour,0,away_team,stadium,away_score,away_w_l]
    
    # if this fails teams stats is empty ex: 290914013
    try:
        table = soup.find_all("table")[1]
    except:
        print(i,game_ids[i],"No Stats")
        continue

    print(i,game_ids[i],stadium,date,hour,lat,long,webpage_response)

    for j,value in enumerate(table.get_text(separator='|').split("|")):
        if j == 67 or j == 68:
            pass
        elif j % 3 == 1:
            for stat in value.split("-"):
                if "." in stat:
                    temp_away.append(float(stat))
                elif ":" in stat:
                    temp_away.append(str(stat))
                else:
                    try:
                        temp_away.append(int(stat))
                    except:
                        temp_away.append(str(stat))
        elif j % 3 == 2:
            for stat in value.split("-"):
                if "." in stat:
                    temp_home.append(float(stat))
                elif ":" in stat:
                    temp_home.append(str(stat))
                else:
                    try:
                        temp_home.append(int(stat))
                    except:
                        temp_home.append(str(stat))
    for stat in weather_lst:
        temp_away.append(stat)
        temp_home.append(stat)

    stats.append(temp_away)
    stats.append(temp_home)
    
    time.sleep(1)
    

IndexError: list index out of range

In [166]:
print(len(stats[0][:-1]))
print("%s,"*len(stats[0]))

47
%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,


In [1]:
cur.execute("DELETE FROM games")
columns = "(id,game_id,date,time,home_away,team,stadium_id,score,winner,1st_downs,passing_1st_downs,rushing_1st_downs,penalty_1st_downs,3rd_down_conversions,3rd_down_attempts,4th_down_conversions,4th_down_attempts,total_plays,total_yards,total_drives,yards_per_play,passing_yards,passing_completions,passing_attempts,yards_per_pass,interceptions_thrown,sacks_lost,yards_lost_to_sacks,rushing_yards,rushing_attempts,yards_per_rush,red_zone_tds,red_zone_attempts,penalties,penalty_yards,turnovers,fumbels_lost,dst_tds,possession_time,temp,humidity,apparent_temp,rain,snow,surface_pressure,cloudcover,wind_speed,wind_direction)"  # Make sure to correctly outline your column names as they appear in your MySQL table
sql = "INSERT INTO games " + columns + " VALUES (" + "%s,"*len(stats[0][:-1]) + "%s)"  # The '%s,'*len(data[0])[:-1] will provide the right amount of %s placeholders for variables in your SQL statement
for row in stats:
    if len(row) == 48:
        print(tuple(row))
        cur.execute(sql, tuple(row))
        mydb.commit()

NameError: name 'cur' is not defined